In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
from splitbert.textsplit import text_segmentation
from splitbert.splitbert import SplitBertConcatEncoderModel
from splitbert.splitbert import conduct_input_ids_and_attention_masks
from splitbert.splitbert import make_masks
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import tqdm

/home/mykim/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2023-11-05 20:33:16.365508: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Data Preparation

In [2]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')
reply_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/avg_satisfaction_raw_0-999.csv', encoding='ISO-8859-1')

modes = [['seg', 'seg', 'snt']]

nlp = English()
nlp.add_pipe("sentencizer")

# satisfaction score (y)
satisfactions_float = list(reply_df['satisfy_composite'])
satisfactions = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)

reply_contents = list(reply_df['replyContent'])
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])


def get_sequences(contents, mode):
    sequences = []

    if mode == 'all':
        for content in contents:
            sequences.append([content])
    elif mode == 'seg':
        for content in contents:
            sentences = list(map(lambda x: str(x), list(nlp(content).sents)))
            sequences.append(text_segmentation(sentences))
    else:  # sentences
        for content in contents:
            sequences.append(list(map(lambda x: str(x), list(nlp(content).sents))))

    return sequences


for mode in modes:
    print(mode)
    post_sequences = get_sequences(post_contents, mode[0])
    comment_sequences = get_sequences(comment_bodies, mode[1])
    reply_sequences = get_sequences(reply_contents, mode[2])

    data = []
    max_post, max_comment, max_reply = 0, 0, 0
    i = 0
    for post, comment, reply, satisfaction, satisfaction_float in zip(post_sequences, comment_sequences,
                                                                          reply_sequences, satisfactions,
                                                                          satisfactions_float):
        if len(post) > max_post:
            max_post = len(post)
        if len(comment) > max_comment:
            max_comment = len(comment)
        if len(reply) > max_reply:
            max_reply = len(reply)

        data.append([i, post, comment, reply, satisfaction, satisfaction_float])
        i += 1

    print(max_post, max_comment, max_reply)
    max_count = max(max_post, max_comment, max_reply)
    print(max_count)

    columns = ['index', 'post_contents', 'comment_contents', 'reply_contents', 'label', 'score']
    df = pd.DataFrame(data, columns=columns)

    # data split (train & test sets)
    idx_train, idx_remain = train_test_split(df.index.values, test_size=0.20, random_state=42)
    idx_val, idx_test = train_test_split(idx_remain, test_size=0.50, random_state=42)

    train_df = df.iloc[idx_train]
    val_df = df.iloc[idx_val]
    test_df = df.iloc[idx_test]

    count_min_label = min(train_df['label'].value_counts())

    labels = [0, 1, 2]

    train_sample_df = pd.DataFrame([], columns=columns)

    for label in labels:
        tmp = train_df[train_df['label'] == label]
        tmp_sampled = tmp.sample(frac=1).iloc[:count_min_label]
        train_sample_df = pd.concat([train_sample_df, tmp_sampled])

    train_sample_df = train_sample_df.sample(frac=1)

['seg', 'seg', 'snt']
10 4 10
10


# Model Preparation

In [3]:
def normalize_tensor(data):
    return (data - torch.min(data)) / (torch.max(data) - torch.min(data))

In [4]:
def forward_func_ig(inputs, p_count, c_count, model, softmax=False):
    embeddings = inputs[0]
    
    for i, now_max, now_count in zip(range(2), [model.max_post_len, model.max_comment_len], [p_count, c_count]):
        now_embeddings = embeddings.swapaxes(0, 1)
        if i == 0:
            now_embeddings = model.pe(now_embeddings[:model.max_post_len])
        else:
            now_embeddings = model.pe(now_embeddings[model.max_post_len:])
            
        # make masks
        src_mask, src_key_padding_mask = make_masks(now_max, now_count, model.device)
        
        encoder_output = model.encoder(now_embeddings, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        encoder_output = torch.mean(encoder_output[:now_count], dim=0).squeeze(0)
        
        if i == 0:
            total_encoder_output = encoder_output
        else:
            total_encoder_output = torch.cat([total_encoder_output, encoder_output])
    
    encoder_outputs = total_encoder_output.unsqueeze(0)
    encoder_outputs = model.classifier1(encoder_outputs)
    encoder_outputs = torch.cat([encoder_outputs, torch.zeros(1, model.embedding_size, device=model.device)], dim=1)

    # mean + flatten attentioned tensor
    batch_embeddings_list = []
    outputs_list = []
    attentions = []
    
    batch_embeddings = inputs.clone().detach()

    for i in range(len(batch_embeddings)):
        non_zero_rows = batch_embeddings[i][0][batch_embeddings[i][0].sum(dim=1) != 0]
        zero_rows = torch.zeros((batch_embeddings[i][0].shape[0] - non_zero_rows.shape[0], model.embedding_size),
                                dtype=torch.int, device=model.device)
        batch_embeddings[i] = torch.cat([non_zero_rows, zero_rows])

    embeddings = batch_embeddings[0].swapaxes(0, 1)
    src_mask, src_key_padding_mask = make_masks(model.max_post_len, [p_count, c_count], model.device,
                                                model.max_post_len, model.max_comment_len, "concat_all")

    encoder_output = model.encoder(embeddings, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
    attention = model.encoder_layer.self_attn(encoder_output, encoder_output, encoder_output,
                                              attn_mask=src_mask, key_padding_mask=src_key_padding_mask)[0]

    if model.attention_mode == 'attention_residual':
        # add & norm
        residual = outputs_list[i]
        attention = self.dropout(attention) + residual
        attention = self.layer_norm(attention)

    # mul mask - diagonal masking
    attention = attention.swapaxes(0, 2)
    mask = torch.tensor([1] * (p_count + c_count) + [0] * (model.max_post_len + model.max_comment_len - (p_count + c_count))).to(model.device)
    attention = attention.mul(mask).swapaxes(0, 2)

    attention = torch.flatten(attention)
    attention = model.attn_classifier1(attention)
    attention = model.attn_classifier2(attention)
    encoder_outputs[i][model.embedding_size:] = attention
    encoder_outputs = model.mean_attn_layer(encoder_outputs)
    logits = model.classifier2(encoder_outputs)
    
    if softmax:
        logits = F.softmax(logits, dim=1)
    
    return logits


ig = IntegratedGradients(forward_func_ig)

In [5]:
def prepare_model(target, path, epoch, softmax):
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device('cpu')

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    model_path = f'{path}/epoch_{epoch}.model'

    if softmax:
        model = SplitBertConcatEncoderModel(num_labels=len(labels), embedding_size=384, max_len=max_count, 
                                            max_post_len=max_post, max_comment_len=max_comment,
                                            device='cpu', target=target, concat_mode="sep_attention",
                                           attention_mode=False, output_attentions=True, softmax=True)
    else:
        model = SplitBertConcatEncoderModel(num_labels=len(labels), embedding_size=384, max_len=max_count, 
                                            max_post_len=max_post, max_comment_len=max_comment,
                                            device='cpu', target=target, concat_mode="sep_attention",
                                           attention_mode=False, output_attentions=True)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to('cpu')
    model.eval()

    for param in model.sbert.parameters():
        param.requires_grad = False
    
    for param in model.bert.parameters():
        param.requires_grad = False
        
    return device, model, tokenizer

In [6]:
def construct_input_ref_pair(targets):
    input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list = [], [], [], []
    
    for contents in targets:
        result = tokenizer(contents, pad_to_max_length=True, truncation=True, max_length=256, return_tensors='pt')
        
        input_ids = result['input_ids']
        sentence_count_list.append(torch.tensor(len(input_ids)).unsqueeze(0))
        attention_masks = result['attention_mask']
        
        pad = (0, 0, 0, max_count-len(input_ids))
        input_ids = nn.functional.pad(input_ids, pad, "constant", 0)
        attention_masks = nn.functional.pad(attention_masks, pad, "constant", 0)
        ref_input_ids = torch.zeros_like(input_ids)

        input_ids_list.append(input_ids.unsqueeze(0))
        ref_input_ids_list.append(ref_input_ids.unsqueeze(0))
        attention_masks_list.append(attention_masks.unsqueeze(0))
    
    return input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list

In [7]:
def summarize_attributions(attribution):
    attributions = attribution.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [8]:
def get_indexes(filename):
    index_df = pd.read_csv(filename, encoding='UTF-8')
    index_df.columns = ['Unnamed: 0', 'prediction', 'label', 'score', 'idx']
    val_index = sorted(list(index_df.idx.values))
    
    return val_index

In [9]:
index_list = get_indexes(f'../predicting-satisfaction-using-graphs/csv/splitbert_classifier/post_comment/seg_seg/epoch_4_result.csv')

In [10]:
def splitbert_integrated_gradient_post_comment(index, post, comment, p_sentences, c_sentences, label, score, visualize=False, softmax=False):
    
    def post_or_comment_or_reply(index):
        for i, sentences in enumerate(all_sentences):
            if all_tokens[index] in sentences:
                if i == 0:
                    return 'post'
                elif i == 1:
                    return 'comment'
                else:
                    return 'reply'
    
    input_ids, ref_input_ids, attention_masks, sentence_counts = construct_input_ref_pair([post, comment])
    
    one_hot_labels = torch.nn.functional.one_hot(torch.tensor(label), num_classes=len(labels))
    inputs = {'labels': one_hot_labels.type(torch.float).to(device),
          'input_ids1': input_ids[0].to(device),
          'input_ids2': input_ids[1].to(device),
          'attention_mask1': attention_masks[0].to(device),
          'attention_mask2': attention_masks[1].to(device),
          'sentence_count1': sentence_counts[0].to(device),
          'sentence_count2': sentence_counts[1].to(device),
          'mode': 'post_comment'
         }
    
    with torch.no_grad():
        inputs = pc_model(**inputs).hidden_states

    # inputs = torch.stack(embeddings, dim=0)
    pred = forward_func_ig(inputs, sentence_counts[0], sentence_counts[1], pc_model)
    # print(f'answer: {label}, predict: {torch.argmax(pred)}')
    
    result = []
    
    attribution, delta = ig.attribute(inputs=inputs, target=torch.argmax(pred), additional_forward_args=(sentence_counts[0], sentence_counts[1], pc_model, softmax), n_steps=1, return_convergence_delta=True)
    attributions = summarize_attributions(attribution)
    f_attributions = torch.flatten(attributions)
    f_attributions = f_attributions[f_attributions.nonzero()].squeeze(1)
    abs_attributions = list(map(abs, map(float, f_attributions)))
    idx_attributions = []
    for j in range(len(abs_attributions)):
        idx_attributions.append((j, abs_attributions[j], f_attributions[j].item()))
    idx_attributions.sort(key=lambda x:x[1], reverse=True)
        
    top3 = idx_attributions[:3]
    
    if visualize:
        all_sentences = [['[[post]]'], post, ['[[comment]]'], comment]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]
        
        vis_attributions = []
        
        j = 0
        for i in range(len(all_tokens)):
            if all_tokens[i] in ['[[post]]', '[[comment]]']:
                vis_attributions.append(0)
            else:
                vis_attributions.append(f_attributions[j].item())
                j += 1
                
        vis_attributions = torch.tensor(vis_attributions)
        
        score_vis = viz.VisualizationDataRecord(vis_attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                f'{label}, {score}',  # true label
                                                p_sentences + ' ' + c_sentences,
                                                vis_attributions.sum(),
                                                all_tokens,
                                                delta)
        raw_text = ' '.join(post) + ' '.join(comment)
        
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])
        print(vis_attributions)
        
    else:
        where = []
        
        all_sentences = [post, comment]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]

        for j in range(len(top3)):
            where.append(post_or_comment_or_reply(top3[j][0]))
            result.append([index, post, comment, score, all_tokens[top3[j][0]], top3[j][2], post_or_comment_or_reply(top3[j][0])])

        # result.append([index, post, comment, score, all_tokens[top3[0][0]], top3[0][2], where[0]])
        # result.append([index, post, comment, score, all_tokens[top3[1][0]], top3[1][2], where[1]])
        # result.append([index, post, comment, score, all_tokens[top3[2][0]], top3[2][2], where[2]])

        return result, label, torch.argmax(pred).item()

In [12]:
device, pc_model, tokenizer = prepare_model('post_comment', '../predicting-satisfaction-using-graphs/splitbert/model/seg_seg/w_softmax', 5, True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
for i in index_list[:1]:
    splitbert_integrated_gradient_post_comment(i, post_sequences[i], comment_sequences[i], post_contents[i], comment_bodies[i], satisfactions[i], satisfactions_float[i], True, True)

 Visualization For Score 


/home/mykim/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"0, 2.45",1 (1.00),"Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first",-0.46,"[[post]] Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... [[comment]] As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first"


tensor([ 0.0000,  0.4364,  0.0000, -0.8998])


In [14]:
device, pc_model, tokenizer = prepare_model('post_comment', '../predicting-satisfaction-using-graphs/splitbert/model/seg_seg/wo_softmax', 4, False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
for i in index_list[:1]:
    splitbert_integrated_gradient_post_comment(i, post_sequences[i], comment_sequences[i], post_contents[i], comment_bodies[i], satisfactions[i], satisfactions_float[i], True, False)

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"0, 2.45",0 (1.00),"Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first",0.68,"[[post]] Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... [[comment]] As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first"


tensor([ 0.0000, -0.2832,  0.0000,  0.9591])
